In [1]:
from utils.elt_config import (
    DB_PATH,
    DataDirectory,
    QueriesDirectory,
    get_cnxn,
    get_info_schema_df,
    main,
)

In [2]:
cnxn = get_cnxn(database=DB_PATH)

In [3]:
data_dir = DataDirectory()
queries = QueriesDirectory()

In [ ]:
info_schema_df = get_info_schema_df(cnxn)
# print("schemas:", SCHEMAS)
print("columns:", list(info_schema_df.columns))
info_schema_df.groupby("schema").size()

In [ ]:
info_schema_df.loc[info_schema_df["schema"] == "main", ["name", "column_names"]].head(
    20
)

In [ ]:
main()